In [ ]:
import os
import sys
import pandas as pd
import warnings
import json
from statsmodels.iolib.smpickle import save_pickle

# --- 1. Configurar o Caminho do Projeto ---
# Adiciona a raiz do projeto ao sys.path para importação de módulos
try:
    # Em um script .py, __file__ está definido.
    current_dir = os.path.dirname(os.path.abspath(__file__))
    project_root = os.path.abspath(os.path.join(current_dir, '..'))
except NameError:
    # Em um ambiente interativo (notebook), __file__ não está definido.
    # Assume-se que o notebook está um nível abaixo da raiz do projeto.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

# --- 2. Importar Funções Personalizadas ---
from Functions.FNC_Pro import configurar_credenciais_bq, Base_venda
from Functions.FNC_SARIMAX import encontrar_melhores_parametros_sarimax, modelo_sarimax

warnings.filterwarnings("ignore")

def pipeline_treinamento_de_modelos(lista_skus, credenciais_path, modelos_output_dir):
    """
    Executa o pipeline de treinamento, encontra os melhores parâmetros e salva
    os modelos e os parâmetros para cada SKU.
    """
    print("--- Iniciando Pipeline de TREINAMENTO de Modelos VMD ---")
    
    # --- Configuração Inicial ---
    configurar_credenciais_bq(credenciais_path)
    os.makedirs(modelos_output_dir, exist_ok=True)
    print(f"Modelos e parâmetros serão salvos em: {modelos_output_dir}")

    exog_vars = [
        'Log_Preco', 'Black_Friday', 'promocionado',
        'Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 
        'Sexta-feira', 'Sábado', 'Domingo'
    ]

    # --- Loop de Treinamento por SKU ---
    for i, sku in enumerate(lista_skus):
        print(f"\n--- Treinando Modelo para SKU {i+1}/{len(lista_skus)}: {sku} ---")
        
        df_venda = Base_venda(sku)

        if df_venda is None or df_venda.empty or len(df_venda) < 60:
            print(f"Dados insuficientes para o SKU {sku}. Pulando.")
            continue

        try:
            best_order, best_seasonal_order, trend = encontrar_melhores_parametros_sarimax(
                df_venda, sku, exog_vars, verbose=True # Ativar verbose para análise
            )
        except Exception as e:
            print(f"Erro fatal ao encontrar parâmetros para SKU {sku}: {e}")
            continue
            
        resultado_modelo = modelo_sarimax(
            df_venda, sku, *exog_vars, 
            order=best_order, 
            seasonal_order=best_seasonal_order, 
            trend=trend,
            verbose=False
        )

        if resultado_modelo:
            try:
                # Salvar os PARÂMETROS em um arquivo JSON
                parametros = {
                    'order': best_order,
                    'seasonal_order': best_seasonal_order,
                    'trend': trend,
                    'exog_vars': exog_vars
                }
                params_path = os.path.join(modelos_output_dir, f'params_sku_{sku}.json')
                with open(params_path, 'w') as f:
                    json.dump(parametros, f)
                
                # Salvar o MODELO treinado em um arquivo .pkl
                model_path = os.path.join(modelos_output_dir, f'model_sku_{sku}.pkl')
                save_pickle(resultado_modelo, model_path)
                
                print(f"Modelo e parâmetros para o SKU {sku} salvos com sucesso.")

            except Exception as e:
                print(f"Erro ao salvar o modelo ou parâmetros para o SKU {sku}: {e}")
        else:
            print(f"Falha ao treinar o modelo para o SKU {sku}.")

    print("\n--- Pipeline de TREINAMENTO Concluído ---")

if __name__ == '__main__':
    caminho_credenciais = r'G:/Drives compartilhados/Bases BI/epoca-230913-b478a9a0dd4c.json'
    
    # Carregar a lista de SKUs do arquivo Excel
    try:
        # O arquivo deve estar na raiz do projeto para ser encontrado
        caminho_skus = os.path.join('C:/Users/joao.pcarvalho/Desktop/Git Repositórios/Forecast-vendas-e-elasticidade/VMD/SKUs_Ativos.xlsx')
        df_skus = pd.read_excel(caminho_skus)
        # Converte a primeira coluna para lista de strings
        skus_para_treinar = df_skus.iloc[:, 0].astype(str).tolist()
        print(f"{len(skus_para_treinar)} SKUs carregados de '{caminho_skus}' para treinamento.")
    except FileNotFoundError:
        print(f"Erro: O arquivo 'SKUs_Ativos.xlsx' não foi encontrado na raiz do projeto: {project_root}")
        # Usar uma lista vazia ou de exemplo para evitar que o script quebre
        skus_para_treinar = []
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo Excel: {e}")
        skus_para_treinar = []

   
    # Pasta para salvar os modelos, dentro da raiz do projeto
    pasta_modelos = os.path.join(project_root, 'Modelos_VMD')

    # Executa o pipeline apenas se a lista de SKUs não estiver vazia
    if skus_para_treinar:
        pipeline_treinamento_de_modelos(skus_para_treinar, caminho_credenciais, pasta_modelos)


In [ ]:
import os
import sys
import pandas as pd
import warnings
import json
from statsmodels.iolib.smpickle import load_pickle

# --- 1. Configurar o Caminho do Projeto ---
# Adiciona a raiz do projeto ao sys.path para importação de módulos
try:
    # Em um script .py, __file__ está definido.
    current_dir = os.path.dirname(os.path.abspath(__file__))
    project_root = os.path.abspath(os.path.join(current_dir, '..'))
except NameError:
    # Em um ambiente interativo (notebook), __file__ não está definido.
    # Assume-se que o notebook está um nível abaixo da raiz do projeto.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

# --- 2. Importar Funções Personalizadas ---
from Functions.FNC_Pro import configurar_credenciais_bq, Base_venda
from Functions.FNC_SARIMAX import gerar_previsoes_vmd

warnings.filterwarnings("ignore")
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 50)

def pipeline_previsao_vmd(lista_skus, credenciais_path, modelos_input_dir):
    """
    Carrega modelos e parâmetros já treinados para gerar previsões de VMD.
    """
    print("--- Iniciando Pipeline de PREVISÃO de VMD ---")
    
    # --- Configuração Inicial ---
    configurar_credenciais_bq(credenciais_path)
    
    resultados_finais = []

    # --- Loop de Previsão por SKU ---
    for i, sku in enumerate(lista_skus):
        print(f"\n--- Gerando Previsão para SKU {i+1}/{len(lista_skus)}: {sku} ---")

        model_path = os.path.join(modelos_input_dir, f'model_sku_{sku}.pkl')
        params_path = os.path.join(modelos_input_dir, f'params_sku_{sku}.json')

        # Verificar se o modelo e os parâmetros existem
        if not os.path.exists(model_path) or not os.path.exists(params_path):
            print(f"Modelo ou parâmetros não encontrados para o SKU {sku}. Pule este SKU.")
            resultados_finais.append({'SKU': sku, 'Status': 'Modelo não treinado'})
            continue

        try:
            # Carregar o modelo e os parâmetros
            modelo_carregado = load_pickle(model_path)
            with open(params_path, 'r') as f:
                parametros = json.load(f)
            
            exog_vars = parametros['exog_vars']
            print(f"Modelo e parâmetros para SKU {sku} carregados com sucesso.")

            # Obter os dados mais recentes
            df_venda = Base_venda(sku)

            if df_venda is None or df_venda.empty:
                print(f"Não foi possível obter dados de venda para o SKU {sku}.")
                resultados_finais.append({'SKU': sku, 'Status': 'Erro ao obter dados'})
                continue

            # Gerar as previsões de VMD
            previsoes = gerar_previsoes_vmd(modelo_carregado, df_venda, sku, exog_vars)
            resultados_finais.append(previsoes)
            print(f"Previsão para SKU {sku} gerada.")

        except Exception as e:
            print(f"Ocorreu um erro ao gerar previsão para o SKU {sku}: {e}")
            resultados_finais.append({'SKU': sku, 'Status': f'Erro: {e}'})

    print("\n--- Pipeline de PREVISÃO Concluído ---")
    return pd.DataFrame(resultados_finais)


if __name__ == '__main__':
    # O caminho correto para o arquivo de credenciais.
    caminho_credenciais = r'G:/Drives compartilhados/Bases BI/epoca-230913-b478a9a0dd4c.json'
    
    # Carregar a lista de SKUs do arquivo Excel
    try:
        # O arquivo deve estar na raiz do projeto
        caminho_skus = os.path.join('C:/Users/joao.pcarvalho/Desktop/Git Repositórios/Forecast-vendas-e-elasticidade/VMD/SKUs_Ativos.xlsx')
        df_skus = pd.read_excel(caminho_skus)
        # Converte a primeira coluna para lista de strings
        skus_para_prever = df_skus.iloc[:, 0].astype(str).tolist()
        print(f"{len(skus_para_prever)} SKUs carregados de '{caminho_skus}' para previsão.")
    except FileNotFoundError:
        print(f"Erro: O arquivo 'SKUs_Ativos.xlsx' não foi encontrado na raiz do projeto: {project_root}")
        skus_para_prever = []
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo Excel: {e}")
        skus_para_prever = []

    # A pasta para carregar os modelos, dentro da raiz do projeto
    pasta_modelos = os.path.join(project_root, 'Modelos_VMD')

    # Executa o pipeline de previsão apenas se a lista de SKUs não estiver vazia
    if skus_para_prever:
        df_previsoes = pipeline_previsao_vmd(skus_para_prever, caminho_credenciais, pasta_modelos)

        # Exibir os resultados
        print("\n--- Resultados Finais da Previsão ---")
        print(df_previsoes)

        # Salvar o resultado
        if df_previsoes is not None and not df_previsoes.empty:
            caminho_saida = os.path.join(project_root, 'previsoes_vmd.csv')
            df_previsoes.to_csv(caminho_saida, index=False)
            print(f"\nPrevisões salvas com sucesso em: {caminho_saida}")
